<a href="https://colab.research.google.com/github/ag192/Asistente_yoga_IA/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.listdir('/content/drive/MyDrive')


['Colab Notebooks',
 'Getting started.pdf',
 'Mi proyecto de vida.pptx',
 'cmd.exe',
 'requerimiento nuevo.gdoc',
 'Formulario sin título (2).gform',
 'FORMULARIO..gform',
 'mi pagina web.rar',
 'Mi video1.mp4',
 'GFPI-F-027_Formato_RegistroSocioeconomico_v3.gdoc',
 'CEDULA.jpg',
 'SOCIO ECONOMICO (1).pdf',
 'SOCIO ECONOMICO.pdf',
 '1005488342_COMFASUCRE.pdf',
 '1005488342_RECIBO_DE_ENERGIA.jpeg',
 'Mi video11.mp4',
 'Mi video4.mp4',
 'Mi videop.mp4',
 'Mi video 1.mp4',
 'mi video 2.mp4',
 'Sin título (2).gdoc',
 'Sin título (1).gdoc',
 'Sin título.gdoc',
 'Documento sin título (2).gdoc',
 'GFPI-F-015_Formato_Compromiso_del_Aprendiz_V2 (1).docx.pdf',
 'Hoja de cálculo sin título.gsheet',
 'EC202111103708.pdf',
 'Documento sin título (1).gdoc',
 'TRABAJO DE AULA PROGRAMACION II (1).gdoc',
 'TRABAJO DE AULA PROGRAMACION II.gdoc',
 'Formulario sin título (1).gform',
 'Exposicion optoaisladores.gdoc',
 'Documento sin título.gdoc',
 'Sql.rar',
 'Jam sin título.pdf',
 'INVENTARIO 

In [ ]:

from google.colab import drive
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del dataset original
base_path = "/content/drive/MyDrive/yoga_dataset"

# Ruta del dataset aumentado
augmented_path = "/content/drive/MyDrive/YogaDataset_Augmented"

# Crear la carpeta de salida si no existe
os.makedirs(augmented_path, exist_ok=True)

# Configurar el generador de aumento
datagen = ImageDataGenerator(
    rotation_range=20,           # rota hasta 20 grados
    width_shift_range=0.1,       # mueve la imagen horizontalmente
    height_shift_range=0.1,      # mueve la imagen verticalmente
    brightness_range=[0.7, 1.3], # ajusta brillo
    shear_range=10,              # distorsión leve
    zoom_range=0.1,              # acerca/aleja ligeramente
    horizontal_flip=True,        # invierte horizontalmente
    fill_mode='nearest'
)

N_AUMENTADAS = 5  #numero de imagenes a duplicar


for pose_folder in os.listdir(base_path):
    pose_path = os.path.join(base_path, pose_folder)
    if not os.path.isdir(pose_path):
        continue

    print(f"📂 Procesando clase: {pose_folder}")


    save_folder = os.path.join(augmented_path, pose_folder)
    os.makedirs(save_folder, exist_ok=True)

    for file in tqdm(os.listdir(pose_path)):
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(pose_path, file)
        img = cv2.imread(img_path)
        if img is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 256))
        img = np.expand_dims(img, axis=0)

        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=save_folder,
                                  save_prefix='aug', save_format='jpg'):
            i += 1
            if i >= N_AUMENTADAS:
                break

print("Aumento de datos completado.")
print(f"Dataset aumentado guardado en: {augmented_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Procesando clase: perro


100%|██████████| 32/32 [00:40<00:00,  1.27s/it]


📂 Procesando clase: arbol


100%|██████████| 36/36 [00:29<00:00,  1.24it/s]

✅ Aumento de datos completado.
📁 Dataset aumentado guardado en: /content/drive/MyDrive/YogaDataset_Augmented


In [4]:
!pip install mediapipe opencv-python pandas


In [5]:
import cv2
import mediapipe as mp
import csv, os
import pandas as pd


mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)


base_dir = "/content/drive/MyDrive/yoga_dataset"
dataset_path = "dataset_yoga_completo.csv"


header = ["pose_name"] + [f"{i}_{axis}" for i in range(33) for axis in ['x', 'y', 'z']]


if os.path.exists(dataset_path):
    data_existente = pd.read_csv(dataset_path)
    carpetas_existentes = set(data_existente['pose_name'].unique())
    print(f"Dataset previo encontrado con {len(data_existente)} registros.")
    print(f"Poses ya incluidas: {carpetas_existentes}")
else:
    data_existente = pd.DataFrame(columns=header)
    carpetas_existentes = set()
    print("⚠️ No se encontró dataset previo. Se creará uno nuevo.")


csv_file = open("dataset_temp.csv", "w", newline="")
writer = csv.writer(csv_file)
writer.writerow(header)


for folder in os.listdir(base_dir):
    if folder in carpetas_existentes:
        print(f" Pose '{folder}' ya existe, se omite.")
        continue

    folder_path = os.path.join(base_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    print(f"Procesando nueva pose: {folder}")

    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(folder_path, img_name)
            image = cv2.imread(img_path)
            if image is None:
                continue

            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                row = [folder]
                for lm in results.pose_landmarks.landmark:
                    row.extend([lm.x, lm.y, lm.z])
                writer.writerow(row)

csv_file.close()

nuevo_data = pd.read_csv("dataset_temp.csv")


dataset_final = pd.concat([data_existente, nuevo_data], ignore_index=True)
dataset_final.to_csv(dataset_path, index=False)

print(" Dataset actualizado correctamente.")
print(f"Total de ejemplos ahora: {len(dataset_final)}")
print(f"Poses incluidas: {dataset_final['pose_name'].unique()}")


Dataset previo encontrado con 87 registros.
Poses ya incluidas: {'perro', 'arbol'}
 Pose 'perro' ya existe, se omite.
 Pose 'arbol' ya existe, se omite.
 Dataset actualizado correctamente.
Total de ejemplos ahora: 87
Poses incluidas: ['perro' 'arbol']


/tmp/ipython-input-550944928.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset_final = pd.concat([data_existente, nuevo_data], ignore_index=True)


In [6]:
import pandas as pd

data = pd.read_csv("dataset_yoga_completo.csv")
print(f"Registros: {len(data)}")
print(f"Columnas: {len(data.columns)}")
data.head()


Registros: 87
Columnas: 100


,pose_name,0_x,0_y,0_z,1_x,1_y,1_z,2_x,2_y,2_z,...,29_z,30_x,30_y,30_z,31_x,31_y,31_z,32_x,32_y,32_z
0,perro,0.611971,0.692306,-0.039784,0.632928,0.700462,-0.023035,0.634662,0.697529,-0.023089,...,0.389977,0.051615,0.858064,0.023426,0.204213,0.869004,0.301396,0.180514,0.894144,-0.106404
1,perro,0.463652,0.738091,0.033072,0.451049,0.749701,0.006250,0.449637,0.747938,0.006380,...,-0.058623,0.756412,0.840748,0.222197,0.682168,0.893987,-0.174329,0.668811,0.876108,0.146113
2,perro,0.493606,0.789608,-0.006361,0.479337,0.795620,-0.029128,0.477720,0.793874,-0.029062,...,-0.008932,0.839930,0.849490,0.234993,0.769768,0.882686,-0.091971,0.765375,0.873732,0.184488
3,perro,0.380776,0.738391,-0.028320,0.357350,0.738002,-0.068460,0.356024,0.733474,-0.068450,...,-0.057023,0.870750,0.872177,0.383441,0.753226,0.935905,-0.225593,0.747058,0.919649,0.291023
4,perro,0.597084,0.618320,0.047911,0.607675,0.630824,0.070844,0.608678,0.630163,0.070790,...,0.062718,0.229197,0.748720,-0.129017,0.328525,0.778619,-0.007124,0.317290,0.794322,-0.233528


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

data = pd.read_csv("dataset_yoga_completo.csv")

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, test_size=0.2, random_state=42
)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(y_categorical.shape[1], activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=8,
    validation_split=0.2,
    verbose=1
)


loss, acc = model.evaluate(X_test, y_test)
print(f"Precisión en test: {acc*100:.2f}%")


model.save("modelo_yoga.h5")
print("💾 Modelo guardado como modelo_yoga.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.4966 - loss: 0.7567 - val_accuracy: 0.5000 - val_loss: 0.6628
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8014 - loss: 0.4698 - val_accuracy: 0.9286 - val_loss: 0.4118
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9515 - loss: 0.3358 - val_accuracy: 0.9286 - val_loss: 0.3042
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9761 - loss: 0.2151 - val_accuracy: 0.9286 - val_loss: 0.2481
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9929 - loss: 0.1280 - val_accuracy: 0.9286 - val_loss: 0.2483
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9897 - loss: 0.0973 - val_accuracy: 0.9286 - val_loss: 0.2757
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9806 - loss: 0.0678 - val_accuracy: 0.9286 - val_loss: 0.3030
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0397 - val_accuracy: 0.9286 - val_loss: 0.3053


Precisión en test: 100.00%
💾 Modelo guardado como modelo_yoga.h5


In [8]:
from google.colab import files
files.download("modelo_yoga.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

dataset_path = "/content/drive/MyDrive/YogaDataset_Augmented"

for folder in sorted(os.listdir(dataset_path)):
    path = os.path.join(dataset_path, folder)
    if os.path.isdir(path):
        num_files = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
        print(f"📁 {folder}: {num_files} imágenes")


📁 arbol: 180 imágenes
📁 perro: 160 imágenes
